In [1]:
# Bibliotecas Necessárias

!pip install -q transformers sentence-transformers langchain langchain-community faiss-cpu # faiss-cpu for CPU, faiss-gpu if you have a powerful NVIDIA GPU and want faster indexing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

In [15]:
# Imports necessários

import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from collections import Counter

In [3]:
# Configuração Inicial

diretorio = "todo_texto_extraido"

diretorio_faiss = "indice_faiss"
os.makedirs(diretorio_faiss, exist_ok=True)

modelo = "sentence-transformers/all-MiniLM-L6-v2"

tam_chunk = 1000
overlap = 100

In [35]:
# Funções Auxiliares

def carregar_textos(diretorio):

  todos_texto = {}
  print(f"\nCarregando textos do '{diretorio}'...")

  if not os.path.exists(diretorio):
    print(f"O diretório '{diretorio}' não existe. Garanta que o caminho está correto.")
    return todos_texto

  for arquivo in os.listdir(diretorio):
    if arquivo.endswith(".txt"):
      caminho_arquivo = os.path.join(diretorio, arquivo)

      try:
        with open(caminho_arquivo, "r", encoding="utf-8") as f:
          texto = f.read()
          todos_texto[arquivo] = texto
          print(f"Texto do arquivo '{arquivo}' carregado com sucesso.")
      except Exception as e:
        print(f"Erro ao carregar o arquivo '{arquivo}': {e}")

  if not todos_texto:
    print(f"Sem .txt no '{diretorio}'")

  return todos_texto

def chunkar_textos(textos_documentos, tam_chunk, overlap):

    divisor_texto = RecursiveCharacterTextSplitter(
        chunk_size=tam_chunk,
        chunk_overlap=overlap,
        separators=["\n\n", "\n", ".", " ", ""],
        add_start_index=True,
    )

    textos_chunks = []
    print("Arquivos detectados:", list(textos_documentos.keys()))

    print(f"\nChunkando textos (chunk_size = {tam_chunk}, chunk_overlap = {overlap})...")

    for arquivo, texto in textos_documentos.items():
        print(f"  DEBUG: Document '{os.path.basename(arquivo)}' length BEFORE chunking: {len(texto)} characters.")
        chunks = divisor_texto.create_documents([texto], metadatas=[{"Fonte": os.path.basename(arquivo)}])
        textos_chunks.extend(chunks)
        print(f"Documento '{os.path.basename(arquivo)}' dividido em {len(chunks)} chunks.")

    if not textos_chunks:
        print("Nenhum chunk gerado. Cheque se os arquivos de texto não estão vazios.")
    else:
        print(f"Total de chunks gerados: {len(textos_chunks)}")
        print("\nExemplo de um chunk gerado:")
        print(textos_chunks[0].page_content[:200] + "...")
        print(f"Metadados: {textos_chunks[0].metadata}")
    return textos_chunks


def criar_indice_faiss(chunks, modelo, indice_caminho):

  print(f"Modelo de embedding: {modelo}...")

  kwargs = {}
  if os.environ.get('COLAB_GPU', False):
    kwargs['device'] = 'cuda'

  else:
    kwargs['device'] = 'cpu'

  embeddings = HuggingFaceEmbeddings(model_name = modelo,
                                     model_kwargs = kwargs)


  print(f"Criando o índice FAISS em '{indice_caminho}'...")
  vetor_sema = FAISS.from_documents(chunks, embeddings)
  vetor_sema.save_local(indice_caminho)
  print("Índice FAISS criado com sucesso!")

In [37]:
def analisar_chunks(textos_chunks):
    contagem = Counter([chunk.metadata["Fonte"] for chunk in textos_chunks])
    print("\nResumo por documento:")
    for nome, count in contagem.items():
            print(f"{nome}: {count} chunks")
    print(f"Total de chunks contados: {sum(contagem.values())}")
    for i, chunk in enumerate(textos_chunkados):
      if chunk.metadata["Fonte"] == "CÓDIGO DE OBRAS.txt":
        print("Chunks do CÓDIGO DE OBRAS.txt:\n")
        print(f"Chunk {i}: {len(chunk.page_content)} caracteres")
      if chunk.metadata["Fonte"] == "tabela.txt":
        print("Chunks do tabela.txt:\n")
        print(f"Chunk {i}: {len(chunk.page_content)} caracteres")

In [39]:
# Carrega os textos extraidos
textos_extraidos = carregar_textos(diretorio)

#Chunka os textos
textos_chunkados = chunkar_textos(textos_extraidos, tam_chunk, overlap)
# analisar_chunks(textos_chunkados)

# Cria o índice FAISS
banco_vetores = criar_indice_faiss(textos_chunkados, modelo, diretorio_faiss)

print("\nIndexão completa!")
print(f"Os indices estão no diretório: '{diretorio_faiss}'")


Carregando textos do 'todo_texto_extraido'...
Texto do arquivo 'CÓDIGO DE OBRAS.txt' carregado com sucesso.
Texto do arquivo 'tabela.txt' carregado com sucesso.
Arquivos detectados: ['CÓDIGO DE OBRAS.txt', 'tabela.txt']

Chunkando textos (chunk_size = 1000, chunk_overlap = 100)...
  DEBUG: Document 'CÓDIGO DE OBRAS.txt' length BEFORE chunking: 204239 characters.
Documento 'CÓDIGO DE OBRAS.txt' dividido em 226 chunks.
  DEBUG: Document 'tabela.txt' length BEFORE chunking: 2500 characters.
Documento 'tabela.txt' dividido em 5 chunks.
Total de chunks gerados: 231

Exemplo de um chunk gerado:
PREFEITURA MUNICIPAL DE EUSÉBIO
Av. Edimilson Pinheiro n.º 150- Autódromo-CE-CEP61.760-000
“PREFEITO E POVO JUNTOS”
PLANO DIRETOR DE DESENVOLVIMENTO URBANO DE EUSÉBIO -
CÓDIGO DE OBRAS, EDIFICAÇÕES E ...
Metadados: {'Fonte': 'CÓDIGO DE OBRAS.txt', 'start_index': 0}
Modelo de embedding: sentence-transformers/all-MiniLM-L6-v2...
Criando o índice FAISS em 'indice_faiss'...
Índice FAISS criado com sucess